# View data

ParamTools provides 2 primary ways for you to view and access your parameter values:

1. Using `.sel`: `params.sel["parameter_name"]`
2. As NumPy arrays: `params.parameter_name`

`.sel` is for querying parameter values based on their labels or values. Accessing values as NumPy arrays is for using parameter values for computational tasks. 

First, let's create some data.

In [1]:
import paramtools


class Params(paramtools.Parameters):
    defaults = {
        "schema": {
            "labels": {
                "date": {
                    "type": "date", 
                    "validators": {
                        "range": {"min": "2020-01-01", "max": "2021-01-01", "step": {"months": 1}}
                    }
                }
            },
        },
        "a": {
            "title": "A",
            "type": "int",
            "value": [{"date": "2020-01-01", "value": 2}, {"date": "2020-10-01", "value": 8},]
        },
        "b": {
            "title": "B",
            "type": "float",
            "value": [{"date": "2020-01-01", "value": 10.5}]
        }
    }


## Access values using `.sel`


In [2]:
params = Params()


View all of the values of `a`:

In [3]:
params.sel["a"]

Values([
  {'date': datetime.date(2020, 1, 1), 'value': 2},
  {'date': datetime.date(2020, 10, 1), 'value': 8},
])

Access parameters directly using `isel`:

In [4]:
params.sel["a"].isel[0]

{'date': datetime.date(2020, 1, 1), 'value': 2}

Access labels and values directly:

In [5]:
params.sel["a"]["date"][0]

datetime.date(2020, 1, 1)

In [6]:
params.sel["a"]["value"][0]

2

ParamTools supports slicing:

In [7]:
params.sel["a"]["date"][:2]

[datetime.date(2020, 1, 1), datetime.date(2020, 10, 1)]

### Use `label_to_extend` to extend a and b's values over multiple months.

In [8]:
params = Params(label_to_extend="date")
params.sel["a"]

Values([
  {'date': datetime.date(2020, 1, 1), 'value': 2},
  {'date': datetime.date(2020, 2, 1), 'value': 2, '_auto': True},
  {'date': datetime.date(2020, 3, 1), 'value': 2, '_auto': True},
  {'date': datetime.date(2020, 4, 1), 'value': 2, '_auto': True},
  {'date': datetime.date(2020, 5, 1), 'value': 2, '_auto': True},
  {'date': datetime.date(2020, 6, 1), 'value': 2, '_auto': True},
  {'date': datetime.date(2020, 7, 1), 'value': 2, '_auto': True},
  {'date': datetime.date(2020, 8, 1), 'value': 2, '_auto': True},
  {'date': datetime.date(2020, 9, 1), 'value': 2, '_auto': True},
  {'date': datetime.date(2020, 10, 1), 'value': 8},
  {'date': datetime.date(2020, 11, 1), 'value': 8, '_auto': True},
  {'date': datetime.date(2020, 12, 1), 'value': 8, '_auto': True},
  {'date': datetime.date(2021, 1, 1), 'value': 8, '_auto': True},
])

Get all values with a date greater than August 1, 2020:

In [9]:
from datetime import date

params.sel["a"]["date"] > date(2020, 8, 1)

QueryResult([
  {'date': datetime.date(2020, 9, 1), 'value': 2, '_auto': True}
  {'date': datetime.date(2020, 10, 1), 'value': 8}
  {'date': datetime.date(2020, 11, 1), 'value': 8, '_auto': True}
  {'date': datetime.date(2020, 12, 1), 'value': 8, '_auto': True}
  {'date': datetime.date(2021, 1, 1), 'value': 8, '_auto': True}
])

Iterate over the query results:

In [10]:
for val in params.sel["a"]["date"] > date(2020, 8, 1):
    print(val)

OrderedDict([('date', datetime.date(2020, 9, 1)), ('value', 2), ('_auto', True)])
OrderedDict([('date', datetime.date(2020, 10, 1)), ('value', 8)])
OrderedDict([('date', datetime.date(2020, 11, 1)), ('value', 8), ('_auto', True)])
OrderedDict([('date', datetime.date(2020, 12, 1)), ('value', 8), ('_auto', True)])
OrderedDict([('date', datetime.date(2021, 1, 1)), ('value', 8), ('_auto', True)])


Convert first two values of the results to a list:

In [11]:
(params.sel["a"]["date"] > date(2020, 8, 1)).isel[:2]

[{'date': datetime.date(2020, 9, 1), 'value': 2, '_auto': True},
 {'date': datetime.date(2020, 10, 1), 'value': 8}]

## Access values directly as numpy arrays

ParamTools uses information about the parameter labels to convert values from a list of dictionaries to a NumPy array. Set `array_first` to `True` for ParamTools to convert parameter values to arrays by default.

In [12]:
params = Params(label_to_extend="date", array_first=True)

params.a

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 8, 8, 8])

In [13]:
params.a * params.b

array([21., 21., 21., 21., 21., 21., 21., 21., 21., 84., 84., 84., 84.])

Select the values of the array:

In [14]:
from datetime import date

arr = params.to_array("a", date=[date(2020, 1, 1), date(2020, 10, 1)])

arr

array([2, 8])

Convert back to a list of dictionaries:

In [15]:
params.from_array("a", arr, date=[date(2020, 1, 1), date(2020, 10, 1)])

[{'date': datetime.date(2020, 1, 1), 'value': 2},
 {'date': datetime.date(2020, 10, 1), 'value': 8}]

Convert parameter values to a dictionary:

In [16]:
param_dict = params.to_dict()

param_dict

{'a': array([2, 2, 2, 2, 2, 2, 2, 2, 2, 8, 8, 8, 8]),
 'b': array([10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5, 10.5,
        10.5, 10.5])}

In [17]:
import pandas as pd

param_dict["date"] = params.label_grid["date"]


df = pd.DataFrame.from_dict(param_dict)
df.set_index("date")

,a,b
date,,
2020-01-01,2,10.5
2020-02-01,2,10.5
2020-03-01,2,10.5
2020-04-01,2,10.5
2020-05-01,2,10.5
2020-06-01,2,10.5
2020-07-01,2,10.5
2020-08-01,2,10.5
2020-09-01,2,10.5


### Miscellaneous

Get parameter names:

In [18]:
params.keys()

odict_keys(['a', 'b'])

Iterate over parameter values:

In [19]:
for key, value in params.items():
    print(key, value)

a [2 2 2 2 2 2 2 2 2 8 8 8 8]
b [10.5 10.5 10.5 10.5 10.5 10.5 10.5 10.5 10.5 10.5 10.5 10.5 10.5]


Dump all parameter data:

In [20]:
params.dump()

{'schema': {'additional_members': {},
  'labels': {'date': {'type': 'date',
    'number_dims': 0,
    'validators': {'range': {'min': '2020-01-01',
      'max': '2021-01-01',
      'step': {'months': 1}}}}},
  'operators': {'uses_extend_func': False,
   'label_to_extend': 'date',
   'array_first': True}},
 'a': OrderedDict([('title', 'A'),
              ('type', 'int'),
              ('number_dims', 0),
              ('value',
               [OrderedDict([('date', '2020-01-01'), ('value', 2)]),
                OrderedDict([('date', '2020-02-01'), ('value', 2)]),
                OrderedDict([('date', '2020-03-01'), ('value', 2)]),
                OrderedDict([('date', '2020-04-01'), ('value', 2)]),
                OrderedDict([('date', '2020-05-01'), ('value', 2)]),
                OrderedDict([('date', '2020-06-01'), ('value', 2)]),
                OrderedDict([('date', '2020-07-01'), ('value', 2)]),
                OrderedDict([('date', '2020-08-01'), ('value', 2)]),
                O